# 🧮 Phase 1 — Bioeconomic LP (Single-Year) + Validator
This notebook solves a single-year LP and validates against **Table 10 & 11** values (Birr/HH/year).

> Edit CSVs in `./data/` to try different assumptions later; re-run the last cells to update results + validation.

In [2]:
import json
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.optimize import linprog
# from caas_jupyter_tools import display_dataframe_to_user
import ace_tools_open as tools

print('Loaded: numpy, pandas, scipy.optimize.linprog')

Loaded: numpy, pandas, scipy.optimize.linprog


In [ ]:
# ========================================
# DATA STRUCTURES (BLUEPRINTS)
# ========================================
# These are like templates that define what information we need to store
# Think of them as forms with specific fields that must be filled out

@dataclass
class HouseholdClass:
    """
    Blueprint for a household group (e.g., poor, medium, rich households)
    
    INPUT FIELDS (what we need to provide):
    - name: The household category name (e.g., "Poor", "Medium", "Rich")
    - n_households: How many households are in this category
    - adult_equiv: Average number of adults (adjusted for children)
    - labor_endowment: Total work-days available per year from family members
    - land_available: Total land owned (in hectares)
    - max_hired_labor: Maximum work-days that can be hired from others
    
    PURPOSE: Stores all the resources each household group has available
    """
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float

@dataclass
class CropParam:
    """
    Blueprint for a crop type (e.g., maize, teff, wheat)
    
    INPUT FIELDS (what we need to provide):
    - name: Crop name (e.g., "Maize", "Teff")
    - calorie_per_kg: Nutritional energy per kilogram
    - yield_per_ha: How many kg produced per hectare of land
    - price_sale: Market price when selling (Birr per kg)
    - seed_cost_per_ha: Cost of seeds per hectare
    - fert_cost_per_ha: Cost of fertilizer per hectare
    - chem_cost_per_ha: Cost of chemicals/pesticides per hectare
    - labor_req_per_ha: Work-days needed per hectare
    
    PURPOSE: Stores all costs, benefits, and requirements for growing each crop
    """
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float

@dataclass
class LivestockParam:
    """
    Blueprint for livestock type (e.g., cattle, sheep, goats)
    
    INPUT FIELDS (what we need to provide):
    - name: Animal type (e.g., "Cattle", "Sheep")
    - price_sale: Market price when selling (Birr per animal)
    - feed_cost_per_unit: Annual feeding cost per animal
    - vet_cost_per_unit: Annual veterinary cost per animal
    - labor_req_per_unit: Work-days needed per animal per year
    
    PURPOSE: Stores all costs, benefits, and requirements for raising each animal type
    """
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float

@dataclass
class PriceParam:
    """
    Blueprint for market prices
    
    INPUT FIELDS:
    - wage: Daily wage rate (Birr per work-day) for hired labor or off-farm work
    
    PURPOSE: Stores the labor market wage rate
    """
    wage: float

@dataclass
class ScenarioShocks:
    """
    Blueprint for "what-if" scenarios (climate change, policy changes, etc.)
    
    INPUT FIELDS (multipliers - 1.0 means no change):
    - yield_multiplier: Crop productivity change (0.8 = 20% decrease, 1.2 = 20% increase)
    - crop_price_multiplier: Market price change for crops
    - wage_multiplier: Labor wage change
    - fert_price_multiplier: Fertilizer price change
    - population_multiplier: Household size change
    
    PURPOSE: Lets us test how changes in external conditions affect farmers
    Example: yield_multiplier=0.7 simulates a 30% drought impact
    """
    yield_multiplier: float = 1.0
    crop_price_multiplier: float = 1.0
    wage_multiplier: float = 1.0
    fert_price_multiplier: float = 1.0
    population_multiplier: float = 1.0

@dataclass
class ModelParams:
    """
    Master container that holds ALL the model data together
    
    INPUT FIELDS:
    - households: Dictionary of all household groups
    - crops: Dictionary of all crop types
    - livestock: Dictionary of all livestock types
    - prices: Market price information
    - min_kcal_per_person_per_day: Minimum calories needed (nutrition constraint)
    - days_per_year: 365 days (for annual calculations)
    
    PURPOSE: Bundles all input data so we can pass it easily to the solver
    """
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

print('Schemas defined.')


Schemas defined.


In [8]:
DATA_DIR = Path('./data')

# Show what's currently in data/ for easy editing
try:
    tools.display_dataframe_to_user('households.csv', pd.read_csv(DATA_DIR / 'households.csv'))
    tools.display_dataframe_to_user('crops.csv', pd.read_csv(DATA_DIR / 'crops.csv'))
    tools.display_dataframe_to_user('livestock.csv', pd.read_csv(DATA_DIR / 'livestock.csv'))
    tools.display_dataframe_to_user('prices.csv', pd.read_csv(DATA_DIR / 'prices.csv'))
    tools.display_dataframe_to_user(
        "observed_table10_11.csv", pd.read_csv(DATA_DIR / "observed_table10_11.csv")
    )
except Exception:
    pass
print('Templates loaded. You can edit any of them and re-run the solver below.')

households.csv


Loading ITables v2.6.2 from the internet... (need help?)


crops.csv


Loading ITables v2.6.2 from the internet... (need help?)


livestock.csv


...

prices.csv


Loading ITables v2.6.2 from the internet... (need help?)


observed_table10_11.csv


Loading ITables v2.6.2 from the internet... (need help?)


Templates loaded. You can edit any of them and re-run the solver below.


In [ ]:
# ========================================
# FUNCTION 1: LOAD DATA FROM FILES
# ========================================
def load_params_from_folder(folder: Path) -> Tuple[ModelParams, ScenarioShocks]:
    """
    Reads all CSV files and JSON scenario from a folder and converts them into our data structures
    
    INPUTS:
    - folder: Path to folder containing households.csv, crops.csv, livestock.csv, 
              prices.csv, and scenario.json
    
    OUTPUTS:
    - Returns TWO things:
      1. ModelParams: All the baseline data (households, crops, livestock, prices)
      2. ScenarioShocks: The scenario multipliers (droughts, price changes, etc.)
    
    HOW IT WORKS:
    1. Read each CSV file into a pandas table
    2. Loop through each row and create our data structure objects
    3. Store everything in dictionaries for easy lookup by name
    4. Read scenario.json for "what-if" multipliers
    """
    # Step 1: Read all CSV files into pandas DataFrames (tables)
    hh = pd.read_csv(folder / 'households.csv')
    crops = pd.read_csv(folder / 'crops.csv')
    livest = pd.read_csv(folder / 'livestock.csv')
    prices_df = pd.read_csv(folder / 'prices.csv')
    
    # Step 2: Read the scenario (what-if conditions) from JSON file
    with open(folder / 'scenario.json', 'r') as f:
        sc_dict = json.load(f)

    # Step 3: Convert household CSV rows into HouseholdClass objects
    # Creates a dictionary: {"Poor": HouseholdClass(...), "Medium": HouseholdClass(...), ...}
    households = {r['name']: HouseholdClass(
        name=r['name'], n_households=float(r['n_households']), adult_equiv=float(r['adult_equiv']),
        labor_endowment=float(r['labor_endowment']), land_available=float(r['land_available']),
        max_hired_labor=float(r.get('max_hired_labor', 0.0))
    ) for _, r in hh.iterrows()}

    # Step 4: Convert crop CSV rows into CropParam objects
    # Creates a dictionary: {"Maize": CropParam(...), "Teff": CropParam(...), ...}
    crops_map = {r['name']: CropParam(
        name=r['name'], calorie_per_kg=float(r['calorie_per_kg']), yield_per_ha=float(r['yield_per_ha']),
        price_sale=float(r['price_sale']), seed_cost_per_ha=float(r['seed_cost_per_ha']),
        fert_cost_per_ha=float(r['fert_cost_per_ha']), chem_cost_per_ha=float(r['chem_cost_per_ha']),
        labor_req_per_ha=float(r['labor_req_per_ha'])
    ) for _, r in crops.iterrows()}

    # Step 5: Convert livestock CSV rows into LivestockParam objects
    # Creates a dictionary: {"Cattle": LivestockParam(...), "Sheep": LivestockParam(...), ...}
    livest_map = {r['name']: LivestockParam(
        name=r['name'], price_sale=float(r['price_sale']), feed_cost_per_unit=float(r['feed_cost_per_unit']),
        vet_cost_per_unit=float(r['vet_cost_per_unit']), labor_req_per_unit=float(r['labor_req_per_unit'])
    ) for _, r in livest.iterrows()}

    # Step 6: Extract wage price from prices CSV
    prices = PriceParam(wage=float(prices_df.iloc[0]['wage']))
    
    # Step 7: Create scenario object from the JSON data
    scenario = ScenarioShocks(**sc_dict)
    
    # Step 8: Bundle everything into ModelParams and return both objects
    params = ModelParams(households=households, crops=crops_map, livestock=livest_map, prices=prices)
    return params, scenario


# ========================================
# FUNCTION 2: CREATE VARIABLE INDEX MAP
# ========================================
def build_index_maps(H, C, L):
    """
    Creates a mapping system to track all decision variables in the optimization
    Think of this as creating a numbered list of all the decisions farmers make
    
    INPUTS:
    - H: List of household groups (e.g., ["Poor", "Medium", "Rich"])
    - C: List of crops (e.g., ["Maize", "Teff", "Wheat"])
    - L: List of livestock (e.g., ["Cattle", "Sheep", "Goats"])
    
    OUTPUTS:
    - idx: A dictionary that maps decisions to position numbers
           Example: ("area", "Poor", "Maize") -> 0 means "land for poor HH growing maize is variable #0"
    - pos: Total number of decision variables
    
    HOW IT WORKS:
    For each household and each crop/livestock, we track these decisions:
    1. "area": How much land to allocate to each crop
    2. "cons": How much crop to consume (eat)
    3. "sold": How much crop to sell at market
    4. "stored": How much crop to store for later
    5. "hired": How many work-days to hire from others
    6. "off_farm": How many work-days to work off-farm
    7. "live_units": How many of each animal to raise
    
    Each combination gets a unique number (position) in our optimization array
    """
    idx = {}  # Dictionary to store the mappings
    pos = 0   # Counter for variable positions
    
    # Variables for LAND ALLOCATION (area planted per crop per household)
    for h in H:
        for c in C: 
            idx[("area", h, c)] = pos
            pos += 1
    
    # Variables for CONSUMPTION (kg eaten per crop per household)
    for h in H:
        for c in C: 
            idx[("cons", h, c)] = pos
            pos += 1
    
    # Variables for SALES (kg sold per crop per household)
    for h in H:
        for c in C: 
            idx[("sold", h, c)] = pos
            pos += 1
    
    # Variables for STORAGE (kg stored per crop per household)
    for h in H:
        for c in C: 
            idx[("stored", h, c)] = pos
            pos += 1
    
    # Variables for HIRED LABOR (work-days hired per household)
    for h in H: 
        idx[("hired", h, None)] = pos
        pos += 1
    
    # Variables for OFF-FARM WORK (work-days working elsewhere per household)
    for h in H: 
        idx[("off_farm", h, None)] = pos
        pos += 1
    
    # Variables for LIVESTOCK (number of animals per type per household)
    for h in H:
        for l in L: 
            idx[("live_units", h, l)] = pos
            pos += 1
    
    return idx, pos


# ========================================
# FUNCTION 3: SOLVE THE OPTIMIZATION PROBLEM
# ========================================
def solve_single_year_lp(params: ModelParams, scenario: ScenarioShocks, hh_subset: Optional[List[str]] = None, calories_floor_override: Optional[float] = None):
    """
    This is the CORE FUNCTION that solves the farm optimization problem
    It finds the best allocation of land, labor, and resources to maximize household income
    while meeting nutrition needs and resource constraints
    
    INPUTS:
    - params: All the baseline data (households, crops, livestock, prices)
    - scenario: The "what-if" multipliers (droughts, price changes, etc.)
    - hh_subset: Optional - which households to include (default: all)
    - calories_floor_override: Optional - custom nutrition requirement
    
    OUTPUTS:
    - A dictionary containing:
      * success: True/False if solution was found
      * status: Message about the solution
      * objective: Total net income maximized
      * by_household: Detailed breakdown of revenues and costs for each household group
    
    HOW IT WORKS (THE OPTIMIZATION):
    This is a LINEAR PROGRAMMING problem with:
    
    OBJECTIVE: Maximize total net income
      = (Crop sales + Livestock sales + Off-farm wages) 
        - (Crop input costs + Livestock costs + Hired labor costs)
    
    SUBJECT TO CONSTRAINTS:
    1. PRODUCTION BALANCE: Production = Consumption + Sales + Storage (for each crop)
    2. LAND LIMIT: Total crop area ≤ Available land (for each household)
    3. LABOR LIMIT: Crop labor + Livestock labor + Off-farm ≤ Family labor + Hired labor
    4. HIRING LIMIT: Hired labor ≤ Maximum allowed
    5. NUTRITION: Total calories consumed ≥ Minimum daily requirement × people × 365 days
    
    The solver (linprog) automatically finds the best combination of all decisions
    """
    
    # Step 1: Determine which households and what options to include
    H = hh_subset if hh_subset is not None else list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())

    # Step 2: Create the variable index map and get total number of decision variables
    idx, nvars = build_index_maps(H, C, L)
    
    # Step 3: Build the OBJECTIVE FUNCTION (what to maximize)
    # Create array of coefficients - negative values are revenues, positive are costs
    c = np.zeros(nvars)

    # Apply scenario multipliers
    ym = scenario.yield_multiplier           # Crop yield change
    pm = scenario.crop_price_multiplier      # Crop price change
    wm = scenario.wage_multiplier            # Wage change
    fm = scenario.fert_price_multiplier      # Fertilizer price change

    # For each household, add costs and revenues to the objective function
    for h in H:
        # COST: Hired labor (positive in objective = minimize)
        c[idx[("hired", h, None)]] += (params.prices.wage * wm)
        
        # REVENUE: Off-farm work (negative in objective = maximize)
        c[idx[("off_farm", h, None)]] += -(params.prices.wage * wm)
        
        for cn in C:
            cp = params.crops[cn]
            # REVENUE: Crop sales (negative = maximize)
            c[idx[("sold", h, cn)]] += -(cp.price_sale * pm)
            
            # COST: Crop inputs per hectare (positive = minimize)
            per_ha_cost = cp.seed_cost_per_ha + fm * cp.fert_cost_per_ha + cp.chem_cost_per_ha
            c[idx[("area", h, cn)]] += per_ha_cost
        
        for l in L:
            lv = params.livestock[l]
            # REVENUE: Livestock sales (negative = maximize)
            c[idx[("live_units", h, l)]] += -(lv.price_sale)
            
            # COST: Livestock feed and vet (positive = minimize)
            c[idx[("live_units", h, l)]] += (lv.feed_cost_per_unit + lv.vet_cost_per_unit)

    # Step 4: Build CONSTRAINTS (the rules that must be obeyed)
    A_eq, b_eq, A_ub, b_ub = [], [], [], []  # Equality and inequality constraint matrices
    bounds = [(0, None) for _ in range(nvars)]  # All variables must be non-negative

    # CONSTRAINT 1: PRODUCTION BALANCE (equality constraints)
    # For each household and crop: Yield × Area = Consumption + Sales + Storage
    for h in H:
        for cn in C:
            row = np.zeros(nvars)
            row[idx[("area", h, cn)]] = params.crops[cn].yield_per_ha * ym  # Production
            row[idx[("cons", h, cn)]] = -1.0   # Consumption
            row[idx[("sold", h, cn)]] = -1.0   # Sales
            row[idx[("stored", h, cn)]] = -1.0 # Storage
            A_eq.append(row)
            b_eq.append(0.0)  # Must equal zero (perfect balance)

    # CONSTRAINT 2: LAND AVAILABILITY (inequality constraints)
    # For each household: Sum of all crop areas ≤ Total land available
    for h in H:
        row = np.zeros(nvars)
        for cn in C: 
            row[idx[("area", h, cn)]] = 1.0
        A_ub.append(row)
        b_ub.append(params.households[h].land_available)

    # CONSTRAINT 3: LABOR AVAILABILITY (inequality constraints)
    # For each household: Crop labor + Livestock labor + Off-farm ≤ Family labor + Hired
    for h in H:
        row = np.zeros(nvars)
        # Labor used by crops
        for cn in C: 
            row[idx[("area", h, cn)]] = params.crops[cn].labor_req_per_ha
        # Labor used by livestock
        for l in L: 
            row[idx[("live_units", h, l)]] = params.livestock[l].labor_req_per_unit
        # Labor supplied off-farm (uses family labor)
        row[idx[("off_farm", h, None)]] = 1.0
        # Labor hired (adds to available labor, so negative)
        row[idx[("hired", h, None)]] = -1.0
        A_ub.append(row)
        b_ub.append(params.households[h].labor_endowment)

    # CONSTRAINT 4: HIRING LIMIT (inequality constraints)
    # For each household: Hired labor ≤ Maximum allowed
    for h in H:
        row = np.zeros(nvars)
        row[idx[("hired", h, None)]] = 1.0
        A_ub.append(row)
        b_ub.append(params.households[h].max_hired_labor)

    # CONSTRAINT 5: NUTRITION REQUIREMENT (inequality constraints)
    # For each household: Total calories consumed ≥ Minimum daily need × people × 365
    for h in H:
        row = np.zeros(nvars)
        for cn in C: 
            # Negative because we move it to left side: -calories ≤ -requirement
            row[idx[("cons", h, cn)]] = -params.crops[cn].calorie_per_kg
        
        # Calculate minimum calories needed for the year
        kcal_floor = (calories_floor_override if calories_floor_override is not None 
                      else params.min_kcal_per_person_per_day)
        kcal_need = kcal_floor * (params.households[h].adult_equiv * scenario.population_multiplier) * params.days_per_year
        
        A_ub.append(row)
        b_ub.append(-kcal_need)  # Negative because constraint is flipped

    # Step 5: SOLVE THE OPTIMIZATION using Linear Programming solver
    # The solver finds values for all decision variables that maximize objective
    # while satisfying all constraints
    res = linprog(c, 
                  A_ub=np.array(A_ub), b_ub=np.array(b_ub),  # Inequality constraints
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),  # Equality constraints
                  bounds=bounds,                              # Non-negativity
                  method='highs')                            # Solver algorithm

    # Step 6: Package the results
    out = {'success': bool(res.success), 'status': res.message, 'objective': None, 'by_household': {}}
    if not res.success:
        return out  # Return early if no solution found

    # Step 7: Extract and organize the solution
    out['objective'] = -res.fun  # Negate because linprog minimizes (we want maximum)
    x = res.x  # The optimal values for all decision variables
    
    # Helper function to get value of a specific variable
    def val(kind, h, k=None):
        return float(x[idx[(kind, h, k)]]) if (kind, h, k) in idx else 0.0

    # Step 8: Calculate detailed revenue and cost breakdown for each household
    for h in H:
        # REVENUES
        rev_crops = sum(val('sold', h, cn) * (params.crops[cn].price_sale * scenario.crop_price_multiplier) 
                       for cn in C)
        rev_livestock = sum(val('live_units', h, l) * params.livestock[l].price_sale 
                           for l in L)
        off = val('off_farm', h)
        rev_off = off * (params.prices.wage * scenario.wage_multiplier)
        
        # COSTS
        cost_crop_inputs = sum(val('area', h, cn) * 
                              (params.crops[cn].seed_cost_per_ha + 
                               scenario.fert_price_multiplier * params.crops[cn].fert_cost_per_ha + 
                               params.crops[cn].chem_cost_per_ha) 
                              for cn in C)
        cost_livestock = sum(val('live_units', h, l) * 
                            (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) 
                            for l in L)
        hired = val('hired', h)
        cost_hired = hired * (params.prices.wage * scenario.wage_multiplier)

        # Store breakdown for this household
        out['by_household'][h] = {
            'revenue_breakdown': {
                'crops': rev_crops, 
                'livestock': rev_livestock, 
                'off_farm_wage': rev_off
            },
            'cost_breakdown': {
                'crop_inputs': cost_crop_inputs, 
                'livestock_costs': cost_livestock, 
                'hired_labor_cost': cost_hired
            },
        }

    return out


In [ ]:
# ========================================
# EXECUTION: LOAD DATA AND SOLVE
# ========================================
# This cell runs the actual optimization

# Step 1: Load all data from CSV files in ./data/ folder
# Returns: params (baseline data) and scenario (what-if multipliers)
params, scenario = load_params_from_folder(Path('./data'))

# Step 2: Solve the optimization problem for all households
# This finds the best farm decisions to maximize income while meeting constraints
demo_results = solve_single_year_lp(params, scenario, hh_subset=None)

# Step 3: Display basic results
print('Status:', demo_results['status'])
print('Success:', demo_results['success'])
print('Objective (net income):', demo_results['objective'])

# Step 4: Create a summary table for easy viewing
# Flatten the nested results into a simple table format
if demo_results['success']:
    rows = []
    for h, hd in demo_results['by_household'].items():
        rows.append({
            'household_class': h,
            'rev_crops': hd['revenue_breakdown']['crops'],
            'rev_livestock': hd['revenue_breakdown']['livestock'],
            'rev_off_farm': hd['revenue_breakdown']['off_farm_wage'],
            'cost_crop_inputs': hd['cost_breakdown']['crop_inputs'],
            'cost_livestock': hd['cost_breakdown']['livestock_costs'],
            'cost_hired_labor': hd['cost_breakdown']['hired_labor_cost'],
        })
    model_df = pd.DataFrame(rows)
    
    # Display the table (if ace_tools is available)
    try:
        tools.display_dataframe_to_user("Model summary (for validator)", model_df)
    except Exception:
        display(model_df)

# Step 5: Save results to a JSON file for later validation
with open('./outputs/demo_results.json', 'w') as f:
    json.dump(demo_results, f, indent=2)
print('Saved results to ./outputs/demo_results.json')


Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True
Objective (net income): 851222.4841269841
Model summary (for validator)


Loading ITables v2.6.2 from the internet... (need help?)


Saved results to ./outputs/demo_results.json


## ✅ Validator: Compare Model vs Observed (Tables 10 & 11)
The file `./data/observed_table10_11.csv` contains values parsed from your tables (Birr/HH/year). Edit it anytime to try different observed targets.

In [ ]:
# ========================================
# VALIDATOR: COMPARE MODEL VS REAL-WORLD DATA
# ========================================
# This cell checks how well our model predictions match actual observed data from Tables 10 & 11
# Helps us understand if the model is realistic and where it might need improvement

# Step 1: Load the model results from the previous run
model_path = Path('./outputs/demo_results.json')
if not model_path.exists():
    raise FileNotFoundError('Model results not found. Run the solver cell first to generate ./outputs/demo_results.json')
with open(model_path, 'r') as f:
    model = json.load(f)
if not model.get('success', False):
    raise RuntimeError('Model run was not successful. Please rerun the solver cell.')

# Step 2: Convert model results into a table format
# Extract revenues and costs for each household class
rows = []
for h, hd in model['by_household'].items():
    rows.append({
        'household_class': h,
        'rev_crops': hd['revenue_breakdown']['crops'],
        'rev_livestock': hd['revenue_breakdown']['livestock'],
        'rev_off_farm': hd['revenue_breakdown']['off_farm_wage'],
        'cost_crop_inputs': hd['cost_breakdown']['crop_inputs'],
        'cost_livestock': hd['cost_breakdown']['livestock_costs'],
        'cost_hired_labor': hd['cost_breakdown']['hired_labor_cost'],
    })
model_df = pd.DataFrame(rows)

# Calculate total income and total costs for model predictions
model_df['income_total'] = model_df['rev_crops'] + model_df['rev_livestock'] + model_df['rev_off_farm']
model_df['cost_total'] = model_df['cost_crop_inputs'] + model_df['cost_livestock'] + model_df['cost_hired_labor']

# Step 3: Load the OBSERVED data from field surveys (Tables 10 & 11)
# This is the real-world data we want to compare against
obs_df = pd.read_csv('./data/observed_table10_11.csv')
obs_df['income_total'] = obs_df['rev_crops'] + obs_df['rev_livestock'] + obs_df['rev_off_farm']
obs_df['cost_total'] = obs_df['cost_crop_inputs'] + obs_df['cost_livestock'] + obs_df['cost_hired_labor']

# Step 4: Merge model and observed data side-by-side for comparison
# Creates columns like 'rev_crops_model' and 'rev_crops_obs'
cols = ['rev_crops','rev_livestock','rev_off_farm','income_total','cost_crop_inputs','cost_livestock','cost_hired_labor','cost_total']
merged = model_df.merge(obs_df[['household_class']+cols], on='household_class', suffixes=('_model','_obs'), how='outer')

# Step 5: Calculate percentage difference between model and observed
# Formula: 100% × (model - observed) / observed
# Positive = model overestimates, Negative = model underestimates
def pct_diff(m, o):
    if pd.isna(m) or pd.isna(o): return float('nan')
    if o == 0: return float('inf') if m != 0 else 0.0
    return 100.0 * (m - o) / abs(o)

# Step 6: Create detailed comparison table
# Shows observed, model, absolute difference, and percentage difference for each metric
rows_comp = []
for _, r in merged.iterrows():
    for k in cols:
        rows_comp.append({
            'household_class': r['household_class'],
            'metric': k,
            'observed': r[f'{k}_obs'],
            'model': r[f'{k}_model'],
            'diff': (r[f'{k}_model'] - r[f'{k}_obs']) if pd.notna(r[f'{k}_model']) and pd.notna(r[f'{k}_obs']) else float('nan'),
            'pct_diff_%': pct_diff(r[f'{k}_model'], r[f'{k}_obs'])
        })
comp_df = pd.DataFrame(rows_comp)

# Display the detailed comparison
try:
    tools.display_dataframe_to_user('Validator — Model vs Observed (by metric)', comp_df)
except Exception:
    display(comp_df)

# Step 7: Calculate summary error metrics
# RMSE (Root Mean Square Error): Average magnitude of errors - lower is better
# MAPE (Mean Absolute Percentage Error): Average percentage error - lower is better
agg = []
for k in cols:
    sub = comp_df[comp_df['metric'] == k].copy()
    sub = sub.replace([np.inf, -np.inf], np.nan).dropna(subset=['observed','model'])
    
    if len(sub) == 0:
        rmse = mape = np.nan
    else:
        # RMSE: sqrt of average squared differences (in Birr units)
        rmse = np.sqrt(np.mean((sub['model'] - sub['observed'])**2))
        
        # MAPE: average of percentage errors (as %)
        mape = np.mean(np.abs((sub['model'] - sub['observed']) / sub['observed'].replace(0, np.nan))) * 100.0
    
    agg.append({'metric': k, 'RMSE': rmse, 'MAPE_%': mape})

agg_df = pd.DataFrame(agg)

# Display summary error metrics
# Lower RMSE and MAPE values indicate better model fit to observed data
try:
    tools.display_dataframe_to_user('Validator — Summary Errors (RMSE, MAPE%)', agg_df)
except Exception:
    display(agg_df)


Validator — Model vs Observed (by metric)


Loading ITables v2.6.2 from the internet... (need help?)


Validator — Summary Errors (RMSE, MAPE%)


Loading ITables v2.6.2 from the internet... (need help?)
